In [20]:
from google.colab import drive
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline  # Use imbalanced-learn's Pipeline

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the correct paths to your dataset
train_path = "/content/drive/MyDrive/fraudTrain.csv"
test_path = "/content/drive/MyDrive/fraudTest.csv"

# Load training data
df_train= pd.read_csv(train_path)
# Load training data
df_test= pd.read_csv(test_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# Step 1: Drop unnecessary columns
df_train.drop(columns=['Unnamed: 0', 'trans_num', 'trans_date_trans_time', 'dob','state'], inplace=True)
df_test.drop(columns=['Unnamed: 0', 'trans_num', 'trans_date_trans_time', 'dob','state'], inplace=True)

In [22]:
# Separate target variable before transformation
y_train = df_train.pop('is_fraud')
y_test = df_test.pop('is_fraud')

In [23]:
# Define categorical and numerical columns
categorical_columns = ['merchant', 'category', 'gender', 'city', 'job']  # Removed high-cardinality columns
numerical_columns = ['amt', 'lat', 'long', 'city_pop', 'merch_lat', 'merch_long']

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=True), categorical_columns),  # Ensure dense output
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numerical_columns)
    ]
)

# Split the data into train and test (Stratified split to maintain the imbalance ratio)
X_train, X_test, y_train, y_test = train_test_split(df_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

# Apply preprocessing to the training data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)  # Apply the same transformations to test data

In [24]:
# Apply SMOTE after preprocessing but before model training
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_preprocessed, y_train)

In [19]:
# Train the model
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(random_state=42)

In [25]:
# Make predictions on the correctly preprocessed test set
y_pred = classifier.predict(X_test_preprocessed)

In [26]:
# Evaluate the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    257834
           1       0.83      0.64      0.72      1501

    accuracy                           1.00    259335
   macro avg       0.92      0.82      0.86    259335
weighted avg       1.00      1.00      1.00    259335

